In [ ]:
%load_ext autoreload
%autoreload 2
from synthpop.recipes.starter6 import Starter
starter_ver='6'

from output_verify import output_verify
from synthpop.synthesizer import synthesize_all, enable_logging 
import os
import pandas as pd
import time
import json
import pickle


outputfolder='/home/da/tools/synthpop/demos/outputs/'
#hh_out=outputfolder+geoname+"_starter%s_"%(starter_ver)+'_households.csv'
#pp_out=outputfolder+geoname+"_starter%s_"%(starter_ver)+'_persons.csv'

In [ ]:
#for whole county

results=[]
def synthesize_counties(counties):
    
    for county in counties:
        hh_out=outputfolder+geoname+"_starter%s_"%(starter_ver)+'_households.csv'
        pp_out=outputfolder+geoname+"_starter%s_"%(starter_ver)+'_persons.csv'
        results=[]
        starter = Starter(os.environ["CENSUS"], "MI", county)
        results=synthesize_all(starter)
        print time.time()-time0
        results[0].index.name='hh_id' #result[0] is hh
        results[0]['serialno'] = results[0]['serialno'].map(lambda x: '%f' % x)
        results[1]['serialno'] = results[1]['serialno'].map(lambda x: '%f' % x)
        #results[0].to_csv(outputfolder+county+"_starter%s_"%('3')+"HHs.csv",float_format='%.f') #float_format, not use scentific, pplies to all fp numbers 
        #could use results['serialno'] = results['serialno'].map(lambda x: '%f' % x) to change output format for serialno only
        results[0].to_csv(hh_out)
        results[1].to_csv(pp_out)
        dfveri=output_verify(results[0],results[1],results[3])
        dfveri[0].to_csv(outputfolder+county+"_compare_starter%s_"%(starter_ver)+"households.csv")
        dfveri[1].to_csv(outputfolder+county+"_compare_starter%s_"%(starter_ver)+"persons.csv")
        
        with open(outputfolder+county+"_starter%s_"%(starter_ver)+'.pkl', 'wb') as f:
            pickle.dump(results[2], f)
    return results
#synthesize_counties(["Livingston County","Oakland County","Washtenaw County","Wayne County","Monroe County","Macomb County"])
res=synthesize_counties(["Macomb County"])


In [ ]:
###for selected BGs
#enable_logging()
starter = Starter(os.environ["CENSUS"], "MI", 'Macomb County')
geoname="problembgs2.csv"
dfbg=pd.read_csv(geoname,index_col=None,dtype=str)
hh_out=outputfolder+geoname+"_starter%s_"%(starter_ver)+'_households.csv'
pp_out=outputfolder+geoname+"_starter%s_"%(starter_ver)+'_persons.csv'
indd=[ind for i, ind in dfbg.iterrows()]
households, persons, fit, marginals = synthesize_all(starter, indexes=indd)
print "##### output", persons.shape[0]

#save outputs to two seperate HH and Persons files

households.to_csv(hh_out)
persons.to_csv(pp_out)

dfveri=output_verify(households,persons,marginals)
dfveri[0].to_csv(outputfolder+geoname+"_compare_starter%s_"%(starter_ver)+"households.csv")
dfveri[1].to_csv(outputfolder+geoname+"_compare_starter%s_"%(starter_ver)+"persons.csv")